In [ ]:
# install sqlite3 - backend store

# !pip install pysqlite3

In [ ]:
### Logging model

### adding Artifac location

import mlflow
import pandas as pd
import numpy as np

from sklearn.linear_model import ElasticNet
from sklearn.model_selection import train_test_split
from sklearn import metrics

import matplotlib.pyplot as plt
import seaborn as sns


def eval_metrics(ground_truth, pred):
    
    rmse = metrics.mean_squared_error(ground_truth, pred, squared=False)
    mae = metrics.mean_absolute_error(ground_truth, pred)
    r2 = metrics.r2_score(ground_truth, pred)
    
    return rmse, mae, r2

df = pd.read_csv("../data/winequality-red.csv")

# Split the data
X = df.drop('quality', axis = 1)
y = df['quality']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=43)
X_train.shape


############ mlflow training

experiment_name = "Experiment-1"
current_run_name = "logging in a new URI - v2"
artifact_location = "/Users/saumyagoyal/JupyterNotebook/MLCon/MLCon_23/1_mlflow/artifact_store2"
backend_store_path = "/Users/saumyagoyal/JupyterNotebook/MLCon/MLCon_23/1_mlflow/backend_store/backend_store.db"

db_uri = "sqlite:///" + backend_store_path

# set the tracking and registry uri

mlflow.set_registry_uri(db_uri)
mlflow.set_tracking_uri(db_uri)

tags = {
    "Demo":"True",
    "Created by": "User1"
}

mlflow.create_experiment(experiment_name,artifact_location)
mlflow.set_experiment(experiment_name)

with mlflow.start_run(run_name = current_run_name):
    
    # Training model
    alpha = 0.1
    l1 = 0.5
    lr = ElasticNet(alpha=alpha, l1_ratio=l1)
    lr.fit(X_train, y_train)
    
    y_pred = lr.predict(X_train)

    rmse, mae, r2 = eval_metrics(y_train, y_pred)
    print(f"Training metrics: \nRMSE:{rmse} \nMAE:{mae} \nR2:{r2}")
    
    mlflow.log_param("Alpha", alpha)
    mlflow.log_param("L1 ratio", l1)
    
    mlflow.log_metric("RMSE", rmse)
    mlflow.log_metric("MAE", mae)
    
    mlflow.set_tags(tags)
    
    mlflow.sklearn.log_model(lr, "Linear regression model - 1")


mlflow server --default-artifact-root /Users/saumyagoyal/JupyterNotebook/MLCon/MLCon_23/1_mlflow/artifact_store2 --backend-store-uri sqlite:////Users/saumyagoyal/JupyterNotebook/MLCon/MLCon_23/1_mlflow/backend_store/new_backend_store.db --host 127.0.0.1 --port 5500 >> server_log.txt

Issues that may arize: Backend store is not out of date
- Backend store is not out of date (happends due to sqlalchemy)
    - solution: run mlflow db upgrade <database_uri>, 
    ```mlflow db upgrade sqlite:////Users/saumyagoyal/JupyterNotebook/MLCon/MLCon_23/1_mlflow/backend_store/new_backend_store.db```
- If the above does not solve the problem then, create a new DB within the same folder
```new_backend_store.db```
- And change the value of backend store variable in the above code, so that it points to the correct database   

### END